In [ ]:
import numpy as np
import matplotlib.pyplot as plt

outp_dir = 'lattice_noise_images/'

%matplotlib inline

In [ ]:
def find_ft(img):
    ft = np.fft.fft(img)
    return np.fft.fftshift(ft)


def find_ift(ft):
    ift = np.fft.ifftshift(ft)
    return np.fft.ifft(ift)


def normalize_img(img):
    img_min = abs(np.min(img))
    img_max = abs(np.max(img))
    img_norm = 255 * (img + img_min) / (img_min + img_max)
    return img_norm.astype(int)


def normalize_to_one(img):
    img = np.array(img)
    img_min = abs(np.min(img))
    img_max = abs(np.max(img))
    img_norm = (img + img_min) / (img_min + img_max)
    return img_norm


def generate_single_octave_plane(width, grid_size, random_values=None, func=None):
    num_cells_x = width // grid_size
    
    if random_values is None:
        random_values = np.random.rand(num_cells_x + 1)

    noise_map = np.zeros(width)

    for x in range(width):
        cell_x = x // grid_size

        local_x = (x % grid_size) / grid_size

        left = random_values[cell_x]
        right = random_values[cell_x + 1]
        
        if func is None:
            smooth_x = local_x
        else:
            smooth_x = func(local_x)

        interp = left * (1 - smooth_x) + right * smooth_x

        noise_map[x] += interp

    return random_values, noise_map


def mesh_plane(x_min, x_max, x_step):
    x_lines = [0]

    for i in range(x_min, x_max, x_step):
        x_lines.append(x_lines[-1] + x_step)

    return x_lines


def qubic_interp(x):
    return x * x * (3 - 2 * x)


def fifth_order_interp(x):
    return 6 * x ** 5 - 15 * x ** 4 + 10 * x ** 3

In [ ]:
pic_size = 128
mesh_size = 16

random_values = np.random.rand(pic_size // mesh_size + 1)

In [ ]:
rand_vals, lin_noise = generate_single_octave_plane(pic_size, mesh_size, random_values=random_values)
rand_vals, qubic_noise = generate_single_octave_plane(pic_size, mesh_size, random_values=random_values, func=qubic_interp)

rand_vals_x = mesh_plane(0, pic_size - 1, mesh_size)
noise_x = mesh_plane(0, pic_size - 1, 1)

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
ax.plot(noise_x, lin_noise)
ax.plot(noise_x, qubic_noise)
ax.scatter(rand_vals_x, rand_vals, marker='.')

ax.set_xticks(rand_vals_x)
ax.set_xticks(noise_x, minor=True)
ax.grid(which='both')

ax.grid(which='minor', alpha=0.2)
ax.grid(which='major', alpha=0.5)
ax.set_xbound(-1, pic_size + 1)
ax.set_ybound(0, 1)

plt.xlabel('X')
plt.ylabel('Y')
plt.savefig(outp_dir + 'types_of_interp.png')

In [ ]:
mesh_size = 32
x_mesh = mesh_plane(0, pic_size - 1, 16)

freq = mesh_size
pers = 0.707
octaves = 3
amps = [pers ** (i + 1) for i in range(octaves)]
harmonics = []
random_values_oct = []


fig = plt.figure(figsize=(10, 10))
for i in range(octaves):
    rv = np.random.rand(pic_size // freq + 1)
    random_values_oct.append(rv)
    
    rand_vals, noise = generate_single_octave_plane(pic_size, freq, random_values=rv)
    noise = [val * amps[i] for val in noise]
    rand_vals = [val * amps[i] for val in rand_vals]
    rand_vals_x = mesh_plane(0, pic_size - 1, freq)
    harmonics.append(noise)
    freq = freq // 2
    
    ax = fig.add_subplot(3, 2, i + 1)
    ax.plot(noise)   
    ax.scatter(rand_vals_x, rand_vals, marker='.')

    ax.set_xticks(x_mesh)
    ax.set_xticks(noise_x, minor=True)
    ax.grid(which='both')

    ax.grid(which='minor', alpha=0.2)
    ax.grid(which='major', alpha=0.5)
    ax.set_xbound(-1, pic_size + 1)
    ax.set_ybound(0)

    plt.xlabel('X')
    plt.ylabel('Y')
  

harmonics = np.sum(harmonics, axis=0)
if np.max(harmonics) > 1:
    harmonics = normalize_to_one(harmonics)
       
scatter_vals = []
for index in rand_vals_x:
    if index == pic_size:
        scatter_vals.append(harmonics[pic_size - 1])
    else:
        scatter_vals.append(harmonics[index])
      
    
ax = fig.add_subplot(3, 2, i + 2)
ax.plot(harmonics)
ax.scatter(rand_vals_x, scatter_vals, marker='.')

ax.set_xticks(x_mesh)
ax.set_xticks(noise_x, minor=True)
ax.grid(which='both')

ax.grid(which='minor', alpha=0.2)
ax.grid(which='major', alpha=0.5)
ax.set_xbound(-1, pic_size + 1)
ax.set_ybound(0, 1)

plt.xlabel('X')
plt.ylabel('Y')
plt.subplots_adjust(wspace=0.25, hspace=0.2)
plt.savefig(outp_dir + 'lin_octave_sum.png')

In [ ]:
mesh_size = 32
x_mesh = mesh_plane(0, pic_size - 1, 16)

freq = mesh_size
pers = 0.707
octaves = 3
amps = [pers ** (i + 1) for i in range(octaves)]
harmonics = []


fig = plt.figure(figsize=(10, 10))
for i in range(octaves):
    rand_vals, noise = generate_single_octave_plane(pic_size, freq, random_values=random_values_oct[i], func=qubic_interp)
    noise = [val * amps[i] for val in noise]
    rand_vals = [val * amps[i] for val in rand_vals]
    rand_vals_x = mesh_plane(0, pic_size - 1, freq)
    harmonics.append(noise)
    freq = freq // 2
    
    ax = fig.add_subplot(3, 2, i + 1)
    ax.plot(noise)   
    ax.scatter(rand_vals_x, rand_vals, marker='.')

    ax.set_xticks(x_mesh)
    ax.set_xticks(noise_x, minor=True)
    ax.grid(which='both')

    ax.grid(which='minor', alpha=0.2)
    ax.grid(which='major', alpha=0.5)
    ax.set_xbound(-1, pic_size + 1)
    ax.set_ybound(0)

    plt.xlabel('X')
    plt.ylabel('Y')
  

harmonics = np.sum(harmonics, axis=0)
if np.max(harmonics) > 1:
    harmonics = normalize_to_one(harmonics)
       
scatter_vals = []
for index in rand_vals_x:
    if index == pic_size:
        scatter_vals.append(harmonics[pic_size - 1])
    else:
        scatter_vals.append(harmonics[index])
      
    
ax = fig.add_subplot(3, 2, i + 2)
ax.plot(harmonics)
ax.scatter(rand_vals_x, scatter_vals, marker='.')

ax.set_xticks(x_mesh)
ax.set_xticks(noise_x, minor=True)
ax.grid(which='both')

ax.grid(which='minor', alpha=0.2)
ax.grid(which='major', alpha=0.5)
ax.set_xbound(-1, pic_size + 1)
ax.set_ybound(0, 1)

plt.xlabel('X')
plt.ylabel('Y')
plt.subplots_adjust(wspace=0.25, hspace=0.2)
plt.savefig(outp_dir + 'cubic_octave_sum.png')

In [ ]:
mesh_size = 32
x_mesh = mesh_plane(0, pic_size - 1, 16)

freq = mesh_size
pers = 0.707
octaves = 3
amps = [pers ** (i + 1) for i in range(octaves)]
harmonics = []


fig = plt.figure(figsize=(10, 10))
for i in range(octaves):
    rand_vals, noise = generate_single_octave_plane(pic_size, freq, random_values=random_values_oct[i], func=fifth_order_interp)
    noise = [val * amps[i] for val in noise]
    rand_vals = [val * amps[i] for val in rand_vals]
    rand_vals_x = mesh_plane(0, pic_size - 1, freq)
    harmonics.append(noise)
    freq = freq // 2
    
    ax = fig.add_subplot(3, 2, i + 1)
    ax.plot(noise)   
    ax.scatter(rand_vals_x, rand_vals, marker='.')

    ax.set_xticks(x_mesh)
    ax.set_xticks(noise_x, minor=True)
    ax.grid(which='both')

    ax.grid(which='minor', alpha=0.2)
    ax.grid(which='major', alpha=0.5)
    ax.set_xbound(-1, pic_size + 1)
    ax.set_ybound(0)

    plt.xlabel('X')
    plt.ylabel('Y')
  

harmonics = np.sum(harmonics, axis=0)
if np.max(harmonics) > 1:
    harmonics = normalize_to_one(harmonics)
       
scatter_vals = []
for index in rand_vals_x:
    if index == pic_size:
        scatter_vals.append(harmonics[pic_size - 1])
    else:
        scatter_vals.append(harmonics[index])
      
    
ax = fig.add_subplot(3, 2, i + 2)
ax.plot(harmonics)
ax.scatter(rand_vals_x, scatter_vals, marker='.')

ax.set_xticks(x_mesh)
ax.set_xticks(noise_x, minor=True)
ax.grid(which='both')

ax.grid(which='minor', alpha=0.2)
ax.grid(which='major', alpha=0.5)
ax.set_xbound(-1, pic_size + 1)
ax.set_ybound(0, 1)

plt.xlabel('X')
plt.ylabel('Y')
plt.subplots_adjust(wspace=0.25, hspace=0.2)
plt.savefig(outp_dir + 'fifth_order_octave_sum.png')

### Spectrum

In [ ]:
mesh_size = 32
x_mesh = mesh_plane(0, pic_size - 1, 16)

freq = mesh_size
pers = 0.707
octaves = 3
amps = [pers ** (i + 1) for i in range(octaves)]
harmonics = []


fig = plt.figure(figsize=(10, 10))
for i in range(octaves):
    rand_vals, noise = generate_single_octave_plane(pic_size, freq, random_values=random_values_oct[i], func=qubic_interp)
    noise = [val * amps[i] for val in noise]
    rand_vals = [val * amps[i] for val in rand_vals]
    rand_vals_x = mesh_plane(0, pic_size - 1, freq)
    harmonics.append(noise)
    freq = freq // 2
    
    ax = fig.add_subplot(3, 2, i + 1)
    ax.plot(noise)   
    ax.scatter(rand_vals_x, rand_vals, marker='.')

    ax.set_xticks(x_mesh)
    ax.set_xticks(noise_x, minor=True)
    ax.grid(which='both')

    ax.grid(which='minor', alpha=0.2)
    ax.grid(which='major', alpha=0.5)
    ax.set_xbound(-1, pic_size + 1)
    ax.set_ybound(0)

    plt.xlabel('X')
    plt.ylabel('Y')
  

harmonics = np.sum(harmonics, axis=0)
if np.max(harmonics) > 1:
    harmonics = normalize_to_one(harmonics)
       
scatter_vals = []
for index in rand_vals_x:
    if index == pic_size:
        scatter_vals.append(harmonics[pic_size - 1])
    else:
        scatter_vals.append(harmonics[index])
      
    
ax = fig.add_subplot(3, 2, i + 2)
ax.plot(harmonics)
ax.scatter(rand_vals_x, scatter_vals, marker='.')

ax.set_xticks(x_mesh)
ax.set_xticks(noise_x, minor=True)
ax.grid(which='both')

ax.grid(which='minor', alpha=0.2)
ax.grid(which='major', alpha=0.5)
ax.set_xbound(-1, pic_size + 1)
ax.set_ybound(0, 1)

plt.xlabel('X')
plt.ylabel('Y')
plt.subplots_adjust(wspace=0.25, hspace=0.2)

In [ ]:
qubic_noise -= np.mean(qubic_noise)
ft = np.abs(find_ft(qubic_noise)) 

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
ax.loglog(ft[pic_size // 2:])

f_curve = [1 / (x ** 0.5) for x in range(1, pic_size // 2)]
ax.loglog(f_curve)
ax.grid(which='both')
ax.set_ybound(10 ** -3, 100)
plt.xlabel('k')
plt.ylabel('АЧХ')
plt.savefig(outp_dir + 'qubic_noise_spectrum.png')

In [ ]:
octaves = normalize_to_one(harmonics)
octaves -= np.mean(octaves)
ft = np.abs(find_ft(octaves)) 

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
ax.loglog(ft[pic_size // 2:])

f_curve = [1 / (x ** 0.5) for x in range(1, pic_size // 2)]
ax.loglog(f_curve)
ax.grid(which='both')
ax.set_ybound(10 ** -3, 100)
plt.xlabel('k')
plt.ylabel('АЧХ')
plt.savefig(outp_dir + 'octave_noise_spectrum.png')

In [ ]:
N = 1000000
filter_ = [1 / (i ** 0.5) for i in range(1, N + 1)]

vals = np.random.randn(N)
vals -= np.mean(vals)

fft = find_ft(vals) * filter_
p_noise = find_ift(fft)
ach = np.abs(fft)

plt.loglog(ach / np.max(ach))
plt.loglog(filter_)

In [ ]:
# Parameters
num_samples = 1024
sampling_rate = 1000  # Hz

# Generate pink noise using the Voss-McCartney algorithm
pink_noise = np.random.randn(num_samples)
pink_noise = np.cumsum(pink_noise)
pink_noise -= np.mean(pink_noise)  # Center the signal

# Calculate the power spectral density
pink_freqs = np.fft.fftfreq(num_samples, d=1/sampling_rate)
pink_spectrum = np.abs(np.fft.fft(pink_noise)) ** 2

# Calculate the 1/f spectrum
frequencies = np.fft.fftfreq(num_samples, d=1/sampling_rate)
frequencies = frequencies[1:num_samples//2]
one_over_f_spectrum = 1 / frequencies

# Create the plot
plt.figure(figsize=(10, 6))
plt.loglog(pink_freqs[:num_samples//2], pink_spectrum[:num_samples//2], label='Pink Noise')
plt.loglog(frequencies, one_over_f_spectrum, label='1/f Noise', linestyle='dashed')

plt.xlabel('Frequency (Hz)')
plt.ylabel('Power')
plt.title('Pink Noise Spectrum vs. 1/f Noise Spectrum')
plt.legend()
plt.grid(True)
plt.show()